In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import pygame
import glob
from sklearn.model_selection import train_test_split


physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
if len(physical_devices) == 0:
    print("Warning: No GPU detected. Running on CPU.")
else:
    print("Running on GPU:", physical_devices)


def load_images(image_paths, label, size=(125, 700)):
    images = []
    labels = []
    for path in image_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, size)
        images.append(img)
        labels.append(label)
    return images, labels


finger_images = glob.glob('dataset/finger/C4/*.png')
no_finger_images = glob.glob('dataset/no_finger/C4/*.png')


print(f"Number of finger images: {len(finger_images)}")
print(f"Number of no_finger images: {len(no_finger_images)}")


finger_images, finger_labels = load_images(finger_images, 1)
no_finger_images, no_finger_labels = load_images(no_finger_images, 0)

all_images = finger_images + no_finger_images
all_labels = finger_labels + no_finger_labels


all_images = np.array(all_images) / 255.0
all_labels = np.array(all_labels)


train_images, temp_images, train_labels, temp_labels = train_test_split(
    all_images, all_labels, test_size=0.3, random_state=42
)
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.33, random_state=42
)


print(f"Train: {len(train_images)} images")
print(f"Validation: {len(val_images)} images")
print(f"Test: {len(test_images)} images")


np.save('train_images.npy', train_images)
np.save('train_labels.npy', train_labels)
np.save('val_images.npy', val_images)
np.save('val_labels.npy', val_labels)
np.save('test_images.npy', test_images)
np.save('test_labels.npy', test_labels)


pygame 2.6.1 (SDL 2.28.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
Num GPUs Available:  0
Number of finger images: 105
Number of no_finger images: 106
Train: 147 images
Validation: 42 images
Test: 22 images


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# چک کردن GPU
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
if len(physical_devices) == 0:
    print("Warning: No GPU detected. Training on CPU.")
else:
    print("Training on GPU:", physical_devices)


train_images = np.load('train_images.npy')
train_labels = np.load('train_labels.npy')
val_images = np.load('val_images.npy')
val_labels = np.load('val_labels.npy')
test_images = np.load('test_images.npy')
test_labels = np.load('test_labels.npy')


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(700, 125, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


with tf.device('/GPU:0'):
    history = model.fit(
        train_images, train_labels,
        epochs=10,
        validation_data=(val_images, val_labels),
        batch_size=16
    )


with tf.device('/GPU:0'):
    test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy:.4f}")


model.save('finger_detection_model.h5')


Num GPUs Available:  0



Epoch 1/10


10/10 [==============================] - 6s 416ms/step - loss: 4.0964 - accuracy: 0.5238 - val_loss: 0.5055 - val_accuracy: 0.8333
Epoch 2/10
10/10 [==============================] - 4s 392ms/step - loss: 0.5670 - accuracy: 0.7823 - val_loss: 0.2632 - val_accuracy: 0.8810
Epoch 3/10
10/10 [==============================] - 4s 428ms/step - loss: 0.2443 - accuracy: 0.8980 - val_loss: 0.2669 - val_accuracy: 0.9048
Epoch 4/10
10/10 [==============================] - 4s 393ms/step - loss: 0.2528 - accuracy: 0.9048 - val_loss: 0.1595 - val_accuracy: 0.9286
Epoch 5/10
10/10 [==============================] - 4s 425ms/step - loss: 0.1716 - accuracy: 0.9524 - val_loss: 0.1111 - val_accuracy: 0.9524
Epoch 6/10
10/10 [==============================] - 4s 383ms/step - loss: 0.3058 - accuracy: 0.8639 - val_loss: 0.1382 - val_accuracy: 0.9762
Epoch 7/10
10/10 [==============================] - 4s 387ms/step - loss: 0.1380 - accuracy: 0.9728 - val_loss: 0.1049 

c:\Users\manam\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pygame


physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
if len(physical_devices) == 0:
    print("Warning: No GPU detected. Running on CPU.")
else:
    print("Running on GPU:", physical_devices)


def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # top-left
    rect[2] = pts[np.argmax(s)]  # bottom-right
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left
    return rect


def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    
    maxWidth, maxHeight = 1000, 700
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped


ip_address = "http://192.168.1.100:8080/video"
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open phone camera")
    exit()


key_regions = [
    (0, 0, 125, 700),   # C4
    (125, 0, 125, 700), # D4
    (250, 0, 125, 700), # E4
    (375, 0, 125, 700), # F4
    (500, 0, 125, 700), # G4
    (625, 0, 125, 700), # A4
    (750, 0, 125, 700), # B4
    (875, 0, 125, 700)  # C5
]
key_names = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5']


model = tf.keras.models.load_model('finger_detection_model.h5')


pygame.mixer.init()
sounds = {name: pygame.mixer.Sound(f'sounds/{name}.aiff') for name in key_names}
last_played = {name: False for name in key_names}


fixed_corners = None
print("Press 's' to fix corners, 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame")
        break
    
    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    
    cv2.imwrite('debug_frame.png', frame)
    cv2.imwrite('debug_edges.png', edges)
    cv2.imshow("Edges", edges)
    
    warped = None
    screenCnt = None
    if fixed_corners is None:
        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
        
        for c in contours:
            if cv2.contourArea(c) < 500:
                continue
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            if len(approx) == 4:
                screenCnt = approx
                break
    
    if fixed_corners is not None:
        screenCnt = fixed_corners
    
    if screenCnt is not None:
        detected = frame.copy()
        cv2.drawContours(detected, [screenCnt], -1, (0, 255, 0), 2)
        cv2.imshow("Detected", detected)
        warped = four_point_transform(frame, screenCnt.reshape(4, 2))
        
        
        for i, (x, y, w, h) in enumerate(key_regions):
            patch = warped[y:y+h, x:x+w]
            patch_resized = cv2.resize(patch, (125, 700)) / 255.0
            patch_resized = np.expand_dims(patch_resized, axis=0)
            with tf.device('/GPU:0'):
                prediction = model.predict(patch_resized, verbose=0)[0]
            label = "Finger" if prediction > 0.5 else "No Finger"
            color = (0, 255, 0) if prediction > 0.5 else (0, 0, 255)
            
            cv2.rectangle(warped, (x, y), (x+w, y+h), color, 1)
            cv2.putText(warped, f"{key_names[i]}: {label}", (x+10, y+20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            
            if prediction > 0.5 and not last_played[key_names[i]]:
                sounds[key_names[i]].play()
                last_played[key_names[i]] = True
            elif prediction <= 0.5:
                last_played[key_names[i]] = False
        
        cv2.imshow("Warped", warped)
    else:
        cv2.imshow("Detected", frame)
        cv2.imshow("Warped", np.zeros((700, 1000, 3), dtype=np.uint8))
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('s') and screenCnt is not None:
        fixed_corners = screenCnt
        print("Corners fixed!")

cap.release()
cv2.destroyAllWindows()

Num GPUs Available:  0
Press 's' to fix corners, 'q' to quit
Corners fixed!
